<a href="https://colab.research.google.com/github/eddiegulay/Swahili-Sement-Classification/blob/main/ASRmzizima.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandas numpy tensorflow==2.10.0 tensorflow-io==0.27.0

In [1]:
!pip install pydub

In [2]:
!git clone https://github.com/eddiegulay/Swahili-transcription.git

Cloning into 'Swahili-transcription'...
remote: Enumerating objects: 22282, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 22282 (delta 0), reused 3 (delta 0), pack-reused 22279
Receiving objects: 100% (22282/22282), 651.71 MiB | 22.31 MiB/s, done.
Resolving deltas: 100% (21/21), done.
Updating files: 100% (22267/22267), done.


In [3]:
import os
import random
from glob import glob
import pandas as pd

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_io as tfio

from pydub import AudioSegment
import io

In [4]:
train_file_path = "Swahili-transcription/data/train_audios/"
test_file_path = "Swahili-transcription/data/test_audios/"

train_dataset = "Swahili-transcription/train.csv"
test_dataset = "Swahili-transcription/test.csv"

In [5]:
train_df = pd.read_csv(train_dataset)
train_df.head()

,audio_ID,path,sentence
0,audio_faa7312a,audio_faa7312a.mp3,huko kwa Wakiroba Mkoa wa Mara
1,audio_643a10c1,audio_643a10c1.mp3,Alingaa katika medani za kisiasa na uongozi nd...
2,audio_5b626e74,audio_5b626e74.mp3,Vitu saba ambavyo kila baba atakuwa.
3,audio_5972c5f3,audio_5972c5f3.mp3,inaonyesha mawaziri wapya ambao wamechukua naf...
4,audio_deebd5b0,audio_deebd5b0.mp3,ee hii pia inatumiwa na kiwanda cha


In [6]:
test_df = pd.read_csv(test_dataset)
test_df.columns.to_list()

['audio_ID', 'path']

In [7]:
# train_df = train_df[:500]
# test_df = test_df[:500]

In [ ]:
import concurrent.futures

train_audio_path = "Swahili-transcription/data/train_audios/"
train_wav_path = "Swahili-transcription/data/train_wavs/"

# Create the train_wavs directory if it doesn't exist
os.makedirs(train_wav_path, exist_ok=True)


def convert_mp3_to_wav(mp3_path, wav_path):
    audio = AudioSegment.from_mp3(mp3_path)
    audio = audio.set_channels(1)  # Convert to mono
    audio = audio.set_frame_rate(16000)  # Resample to 16kHz
    audio.export(wav_path, format="wav")

# Set the maximum number of threads
max_threads = 1000

with concurrent.futures.ThreadPoolExecutor(max_workers=max_threads) as executor:
    futures = []

    for index, row in train_df.iterrows():
        filename = row['path']
        if filename.endswith(".mp3"):
            mp3_path = os.path.join(train_audio_path, filename)
            wav_filename = filename.replace(".mp3", ".wav")
            wav_path = os.path.join(train_wav_path, wav_filename)

            future = executor.submit(convert_mp3_to_wav, mp3_path, wav_path)
            futures.append(future)

    # Wait for all threads to finish
    concurrent.futures.wait(futures)

print("Conversion complete. WAV files saved in train_wavs directory.")


In [ ]:
# Create a list to store dictionaries
audio_files = []
text_sentences = train_df['sentence'].tolist()

def process_audio(audio_name, sentence):
    audio_path = os.path.join(train_wav_path, audio_name.replace(".mp3", ".wav"))

    audio_files.append(audio_path)
    text_sentences(sentence)


# Iterate through your dataset and call the function for each entry
for index, row in train_df.iterrows():
  try:
    audio_name = row['path']
    sentence = row['sentence']
    process_audio(audio_name, sentence) # data in sentence paramenter is not working somehow
  except:
    pass

In [ ]:
audio_files[0]

In [ ]:
text_sentences[0]

In [ ]:
len(text_sentences) == len(audio_files)

In [ ]:
def path_to_spectrogram(path):
    # spectrogram using stft
    audio = tf.io.read_file(path)
    audio, _ = tf.audio.decode_wav(audio, 1)
    audio = tf.squeeze(audio, axis=-1)
    stfts = tf.signal.stft(audio, frame_length=200, frame_step=80, fft_length=256)
    x = tf.math.pow(tf.abs(stfts), 0.5)
    # normalisation
    means = tf.math.reduce_mean(x, 1, keepdims=True)
    stddevs = tf.math.reduce_std(x, 1, keepdims=True)
    x = x = (x - means) / (stddevs + 1e-8)
    audio_len = tf.shape(x)[0]
    # padding to 10 seconds
    pad_len = 2754
    paddings = tf.constant([[0, pad_len], [0, 0]])
    x = tf.pad(x, paddings, "CONSTANT")[:pad_len, :]
    return x

In [ ]:
specto = path_to_spectrogram(audio_files[0])
specto

In [ ]:
class VectorizeChar:
    def __init__(self, max_len=50):
        self.vocab = (
            ["-", "#", "<", ">"]
            + [chr(i + 96) for i in range(1, 27)]
            + [" ", ".", ",", "?"]
        )
        self.max_len = max_len
        self.char_to_idx = {}
        for i, ch in enumerate(self.vocab):
            self.char_to_idx[ch] = i

    def __call__(self, text):
        text = text.lower()
        text = text[: self.max_len - 2]
        text = "<" + text + ">"
        pad_len = self.max_len - len(text)
        return [self.char_to_idx.get(ch, 1) for ch in text] + [0] * pad_len

    def get_vocabulary(self):
        return self.vocab

In [ ]:
max_target_len = 200
vactorizer = VectorizeChar(max_target_len)
print("vocab size", len(vactorizer.get_vocabulary()))

In [ ]:
def create_text_ds(texts_list):
    texts = texts_list
    text_ds = [vactorizer(t) for t in texts]
    text_ds = tf.data.Dataset.from_tensor_slices(text_ds)
    return text_ds

In [ ]:
def create_audio_ds(audio_files_list):
    flist = audio_files_list
    audio_ds = tf.data.Dataset.from_tensor_slices(flist)
    audio_ds = audio_ds.map(
        path_to_spectrogram, num_parallel_calls=tf.data.AUTOTUNE
    )
    return audio_ds

In [ ]:
def create_tf_dataset(data, bs=4):
    audio_ds = create_audio_ds(data)
    text_ds = create_text_ds(data)
    ds = tf.data.Dataset.zip((audio_ds, text_ds))
    ds = ds.map(lambda x, y: {"source": x, "target": y})
    ds = ds.batch(bs)
    ds = ds.prefetch(tf.data.AUTOTUNE)
    return ds

In [ ]:
# train test data
train_size = .9
split_idx = int(train_size * len(audio_files))

train_audios = audio_files[:split_idx]
test_audios = audio_files[split_idx:]

train_texts = text_sentences[:split_idx]
test_texts = text_sentences[split_idx:]

In [ ]:
train_text_ds, test_text_ds = create_text_ds(train_texts), create_text_ds(test_texts)

In [ ]:
train_audio_ds, test_audio_ds = create_audio_ds(train_audios), create_audio_ds(test_audios)

In [ ]:
def create_tf_dataset(audio_ds, text_ds, bs=4):
    ds = tf.data.Dataset.zip((audio_ds, text_ds))
    ds = ds.map(lambda x, y: {"source": x, "target": y})
    ds = ds.batch(bs)
    ds = ds.prefetch(tf.data.AUTOTUNE)
    return ds

In [ ]:
ds, val_ds = create_tf_dataset(train_audio_ds, train_text_ds, bs=64), create_tf_dataset(test_audio_ds, test_text_ds, bs=4)

In [ ]:
ds

In [ ]:
val_ds